
# 📘 **Notebook: STAC Agent – Single-Turn Debug**

---

## 🧭 **1. Setup**

This notebook is for debugging to check:
* Agent behavior
* Tool calls (resolve_aoi → search_satellite_scenes)



### **1.1 Import & Runner Setup**

In [ ]:

import sys
from pathlib import Path

PROJECT_ROOT = Path(".").resolve().parents[2]
print(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT / "src"))
from capstone.agent.stac_agent_adk import create_runner_async
from google.genai import types


/home/kimura/work/google5days/capstone


In [ ]:
# runner, session_service = await create_runner_async()
runner, sess = await create_runner_async(
    user_id="kimura",
    session_id="test_case_1",
)




## 🛰️ **2. Helper Functions**

### **2.1 run_query_simple(query)**

In [ ]:
def run_query_simple(query: str):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id="kimura",
        session_id="test_case_1",
        new_message=content,
    )
    for ev in events:
        if ev.is_final_response():
            print(ev.content.parts[0].text)



### **2.2 run_query_with_trace(query)**

→ Display all tool call traces

In [ ]:
def run_query_with_trace(query: str):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id="xxxxx",
        session_id="test_case_2",
        new_message=content,
    )
    for ev in events:
        print("---- EVENT ----")
        print(ev)
    return events



## 🔍 **3. Quick Smoke Test**

### 3.1 LLM の最終応答のみ


In [ ]:
run_query_simple("Find cloud-free Sentinel-2 images over eastern Hokkaido in summer 2023.")



### 3.2 トレースを確認（resolve_aoi → search_satellite_scenes）

In [ ]:
# runner, session_service = await create_runner_async()
runner, sess = await create_runner_async(
    user_id="xxxxx",
    session_id="test_case_2",
)
events = run_query_with_trace(
    "Find cloud-free Sentinel-2 images over eastern Hokkaido in summer 2023."
)

---- EVENT ----
model_version='gemini-2.0-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location_hint': 'eastern Hokkaido'
        },
        id='adk-5cc2c4ec-f8e2-4852-b238-d04ec40395c6',
        name='resolve_aoi'
      )
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=9,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=9
    ),
  ],
  prompt_token_count=5005,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=5005
    ),
  ],
  total_token_count=5014
) live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=-0.29807795418633354 l

---- EVENT ----
model_version='gemini-2.0-flash' content=Content(
  parts=[
    Part(
      text="""I will search for Sentinel-2 images over eastern Hokkaido (bbox: [143.0, 42.5, 146.0, 45.5]) during summer 2023 (interpreted as June 1, 2023 to August 31, 2023) with cloud cover below 10%.

"""
    ),
    Part(
      function_call=FunctionCall(
        args={
          'bbox': [
            143,
            42.5,
            146,
            45.5,
          ],
          'cloud_cover_max': 10,
          'collections': [
            'sentinel-2-l2a',
          ],
          'datetime_range': '2023-06-01T00:00:00Z/2023-08-31T23:59:59Z',
          'limit': 5
        },
        id='adk-02786155-3dfb-4663-96b3-ed54e3d858a3',
        name='search_satellite_scenes'
      )
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateCont


#### 期待されるトレースの形

* resolve_aoi ツールが最初に呼ばれる
* search_satellite_scenes が 2 番目に呼ばれる
* 最後に LLM が応答メッセージを生成

---

## 🧪 **4. Additional Tests**

以下のようなクエリで動作確認できます。

### 4.1 Tokyo area の test


In [ ]:
run_query_with_trace("Show me Sentinel-2 images over the Tokyo area in August 2023 with cloud cover below 10%.")


---- EVENT ----
model_version='gemini-2.0-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location_hint': 'Tokyo area'
        },
        id='adk-004f4597-9e28-4c22-a851-edd15994ad55',
        name='resolve_aoi'
      )
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=9,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=9
    ),
  ],
  prompt_token_count=5381,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=5381
    ),
  ],
  total_token_count=5390
) live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=2.3832803385125266e-07 logpr

---- EVENT ----
model_version='gemini-2.0-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'bbox': [
            138.8,
            34.8,
            140,
            36.2,
          ],
          'cloud_cover_max': 10,
          'collections': [
            'sentinel-2-l2a',
          ],
          'datetime_range': '2023-08-01T00:00:00Z/2023-08-31T23:59:59Z',
          'limit': 5
        },
        id='adk-9012e8f5-5ceb-4d38-9a9c-dfe7fa7be430',
        name='search_satellite_scenes'
      )
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=70,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=70
    ),
  ],
  prompt_token_count=5429,
  prompt_

<generator object Runner.run at 0x7faba05209c0>



### 4.2 情報不足（ツールを呼ばず質問し返す）

In [ ]:

run_query_with_trace("Show me cloud-free images.")

---- EVENT ----
model_version='gemini-2.0-flash' content=Content(
  parts=[
    Part(
      text='Which area and which time period are you interested in?'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=11,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=11
    ),
  ],
  prompt_token_count=6981,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=6981
    ),
  ],
  total_token_count=6992
) live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=-1.509271731430834e-06 logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='e-3121c130-e1a4-4d2b-bdaa-3f109c86c65c' autho

<generator object Runner.run at 0x7fab7b9cee90>


## 📌 **5. ツール呼び出しの抽出ユーティリティ**（Optional）

ツール引数だけ欲しい場合：

In [ ]:

def extract_tool_calls(events):
    calls = []
    for ev in events:
        if hasattr(ev, "tool_calls") and ev.tool_calls:
            for c in ev.tool_calls:
                calls.append({
                    "name": c.name,
                    "args": c.arguments,
                    "id": c.id,
                })
    return calls

tool_calls = extract_tool_calls(events)
tool_calls

[]


## 🎯 **Notebook まとめ**

この Notebook を使うと：

* エージェントが resolve_aoi → search_satellite_scenes の順でツールを呼んでいるか
* LLM がどう引数を構築しているか
* 曖昧クエリの分岐（聞き返し）ができているか
* プロンプトの調整箇所が見える

という一連の動作確認ができます。

---

必要なら：

* evalset と比較するための DataFrame 化
* 差分検出コード
* プロンプト修正方針の整理

も続けて提供できます。
